# 01/20

%matplotlib inline

In [1]:
description = 'train_mobilenet_from_pretrain_v3_norm_val_aro'

In [2]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [3]:
import tqdm
import json

In [4]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath

#train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')
#train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath
#train_df = train_df.append(train_df_2)
#del train_df_2

train_df = train_df[train_df['have_facemesh']]

In [5]:
def scale(x):
    if x > -1 and x < 1:
        return x
    elif x <= -1:
        return -1
    else:
        return 1

In [6]:
train_df.valence = train_df.valence.apply(scale)
train_df.arousal = train_df.arousal.apply(scale)

In [7]:
from eason_utils import DataFrameBatchIterator
from eason_utils import lprint, now_time_string, log_file_name, change_log_file_name

change_log_file_name(f'''{log_file_name.replace('.log', '')}_{description}.log''')

log_file_name='01_22_23:58:55.log'


In [8]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [9]:
mobilenet_pretrained = tf.keras.models.load_model('../models/affectnet_emotions/mobilenet_7.h5')

In [10]:
mobilenet_output = mobilenet_pretrained.get_layer("global_pooling").output

In [11]:
hidden_layers = [256]

In [12]:
valaence_feat = mobilenet_output
for size in hidden_layers:
    valence_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_valence')(valaence_feat)
outputs_valence = (tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_valence')(valence_feat) * 2) - 1

arousal_feat = mobilenet_output
for size in hidden_layers:
    arousal_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_arousal')(arousal_feat)
outputs_arousal = (tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_arousal')(arousal_feat) * 2) - 1

emotion_feat = mobilenet_output
for size in hidden_layers:
    emotion_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_emotion')(emotion_feat)
emotions_count = len(np.unique(train_df.expression))
outputs_emotion = tf.keras.layers.Dense(emotions_count, activation = 'softmax', name = 'outputs_emotion')(emotion_feat)

In [13]:
model = tf.keras.Model(inputs=mobilenet_pretrained.input,
                       outputs=(outputs_valence, outputs_arousal, outputs_emotion) , name="mobilenet_train")

In [14]:
model.summary()

Model: "mobilenet_train"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 225, 225, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 32  864         ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                    

In [15]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [16]:
batch_size = 32
epochs = 16

logs = []

In [ ]:
from telegram_notifier import send_message
import time
start_time = time.time()


for epoch in range(epochs):
    total_loss = 0
    total_step = 0
    total_emotion_correct = 0
    total_valence_loss = 0
    total_arousal_loss = 0
    total_emotion_loss = 0
    total_process = 0
    for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

        imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
            cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))
        img_array = np.array(list(imgs))

        y_valence = np.array(row.valence)
        y_arousal = np.array(row.arousal)
        y_emotion = np.array(row.expression)

        with tf.GradientTape() as tape:
            logits = model(img_array, training=True)

            pred_valence = logits[0]
            pred_arousal = logits[1]
            pred_emotion = logits[2]

            valence_loss = MSE_loss(y_valence, pred_valence)
            arousal_loss = MSE_loss(y_arousal, pred_arousal)
            emotion_loss = SCC_loss(y_emotion, pred_emotion)

            loss = valence_loss + arousal_loss + emotion_loss

            grads = tape.gradient(loss, model.trainable_weights)

            optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += float(loss)
        total_step += 1
        
        total_process += len(row)

        valence_loss = float(valence_loss)
        arousal_loss = float(arousal_loss)
        emotion_loss = float(emotion_loss)
        
        total_valence_loss += valence_loss
        total_arousal_loss += arousal_loss
        total_emotion_loss += emotion_loss
        
        emotion_correct = int(sum(pred_emotion.numpy().argmax(axis = 1) == y_emotion))
        total_emotion_correct += emotion_correct
        
        log = {
            'emotion_correct': emotion_correct,
            'valence_loss': valence_loss,
            'arousal_loss': arousal_loss,
            'emotion_loss': emotion_loss,
            'time_take': time.time() - start_time
        }
        log = json.dumps(log)
        lprint(log)
    save_model_path = f"models/{now_time_string}_{description}_epoch{epoch}_batch_{batch_size}"
    model.save(save_model_path)
    lprint(f"Save {save_model_path}")
    send_message(f"Save {save_model_path}\nepoch {epoch} is finish")
    log = {
        'model_path': save_model_path,
        'total_emotion_correct': total_emotion_correct,
        'total_loss': total_loss,
        'total_valence_loss': total_valence_loss,
        'total_arousal_loss': total_arousal_loss,
        'total_emotion_loss': total_emotion_loss,
        'total_process': total_process,
        'time_take': time.time() - start_time
    }
    log = json.dumps(log)
    lprint(log)

{"emotion_correct": 6, "valence_loss": 0.6175317764282227, "arousal_loss": 0.502682089805603, "emotion_loss": 2.4452576637268066, "time_take": 5.170595645904541}
{"emotion_correct": 10, "valence_loss": 0.6623479127883911, "arousal_loss": 0.5841599702835083, "emotion_loss": 2.0520167350769043, "time_take": 5.710772752761841}
{"emotion_correct": 15, "valence_loss": 0.7335364818572998, "arousal_loss": 0.529263973236084, "emotion_loss": 1.6975122690200806, "time_take": 6.031900644302368}
{"emotion_correct": 16, "valence_loss": 0.6080062389373779, "arousal_loss": 0.45989662408828735, "emotion_loss": 1.6861851215362549, "time_take": 6.608748435974121}
{"emotion_correct": 17, "valence_loss": 0.4788080155849457, "arousal_loss": 0.24722829461097717, "emotion_loss": 1.4941673278808594, "time_take": 7.150410175323486}
{"emotion_correct": 18, "valence_loss": 0.4297572672367096, "arousal_loss": 0.323874831199646, "emotion_loss": 1.4582022428512573, "time_take": 7.657580852508545}
{"emotion_correct"

{"emotion_correct": 18, "valence_loss": 0.4493337869644165, "arousal_loss": 0.3531090021133423, "emotion_loss": 1.4479446411132812, "time_take": 25.052214860916138}
{"emotion_correct": 15, "valence_loss": 0.42381536960601807, "arousal_loss": 0.36100974678993225, "emotion_loss": 1.3806114196777344, "time_take": 25.364317893981934}
{"emotion_correct": 22, "valence_loss": 0.4188275933265686, "arousal_loss": 0.24419096112251282, "emotion_loss": 1.0305546522140503, "time_take": 25.748653411865234}
{"emotion_correct": 16, "valence_loss": 0.4198974370956421, "arousal_loss": 0.22820638120174408, "emotion_loss": 1.5567190647125244, "time_take": 26.014540910720825}
{"emotion_correct": 19, "valence_loss": 0.3952842056751251, "arousal_loss": 0.2494400143623352, "emotion_loss": 1.0072712898254395, "time_take": 26.336581468582153}
{"emotion_correct": 13, "valence_loss": 0.4106682538986206, "arousal_loss": 0.258670836687088, "emotion_loss": 1.5673863887786865, "time_take": 26.674412727355957}
{"emoti

{"emotion_correct": 19, "valence_loss": 0.3145832419395447, "arousal_loss": 0.1985020637512207, "emotion_loss": 1.3720393180847168, "time_take": 41.89852952957153}
{"emotion_correct": 15, "valence_loss": 0.4633387625217438, "arousal_loss": 0.2880898714065552, "emotion_loss": 1.5521538257598877, "time_take": 42.30435252189636}
{"emotion_correct": 16, "valence_loss": 0.3970843255519867, "arousal_loss": 0.2858850955963135, "emotion_loss": 1.5226973295211792, "time_take": 42.652233362197876}
{"emotion_correct": 21, "valence_loss": 0.44568607211112976, "arousal_loss": 0.27560845017433167, "emotion_loss": 1.0719122886657715, "time_take": 43.01363396644592}
{"emotion_correct": 14, "valence_loss": 0.39333105087280273, "arousal_loss": 0.25674688816070557, "emotion_loss": 1.6449885368347168, "time_take": 43.44976043701172}
{"emotion_correct": 14, "valence_loss": 0.3417212665081024, "arousal_loss": 0.2686462998390198, "emotion_loss": 1.6770306825637817, "time_take": 43.80929183959961}
{"emotion_c

{"emotion_correct": 22, "valence_loss": 0.3687835931777954, "arousal_loss": 0.2428264021873474, "emotion_loss": 1.2218945026397705, "time_take": 62.5468053817749}
{"emotion_correct": 15, "valence_loss": 0.40004050731658936, "arousal_loss": 0.3103489279747009, "emotion_loss": 1.729948878288269, "time_take": 62.92239689826965}
{"emotion_correct": 21, "valence_loss": 0.4928288161754608, "arousal_loss": 0.3469313979148865, "emotion_loss": 1.214195728302002, "time_take": 63.32677865028381}
{"emotion_correct": 24, "valence_loss": 0.4174107313156128, "arousal_loss": 0.20820626616477966, "emotion_loss": 0.974003255367279, "time_take": 63.76195549964905}
{"emotion_correct": 22, "valence_loss": 0.5472427010536194, "arousal_loss": 0.24946391582489014, "emotion_loss": 1.0345938205718994, "time_take": 64.2902147769928}
{"emotion_correct": 20, "valence_loss": 0.4022284150123596, "arousal_loss": 0.24529996514320374, "emotion_loss": 1.1204383373260498, "time_take": 64.61559414863586}
{"emotion_correct